In [2]:
import numpy as np

def van_albada_limiter(phi_u, phi_c, phi_d):
    # phi_u: upstream, phi_c: center, phi_d: downstream
    theta = (phi_c - phi_u) / (phi_d - phi_c + 1e-12)
    return (theta**2 + theta) / (theta**2 + 1.0 + 1e-12)

def flux_limited(phi, u_face):
    """Apply van Albada limiter to the variable phi at cell face."""
    # phi: array of values at cell centers
    N = len(phi)
    phi_face = np.zeros(N-1)
    for i in range(1, N-1):
        # Upwind, center, downstream
        if u_face[i] >= 0:
            phi_u, phi_c, phi_d = phi[i-1], phi[i], phi[i+1]
        else:
            phi_u, phi_c, phi_d = phi[i+2], phi[i+1], phi[i]
        l = van_albada_limiter(phi_u, phi_c, phi_d)
        phi_face[i] = phi_c + 0.5 * l * (phi_d - phi_c)
    # Fill boundaries (simple copy)
    phi_face[0] = phi[0]
    phi_face[-1] = phi[-1]
    return phi_face

# -- Geometry, physical parameters --
N = 200
L = 10.0
dx = L / N
r = 0.05  # pipe radius
A_total = np.pi * r**2

# Initial conditions
rho_L = np.ones(N) * 1000.0
rho_G = np.ones(N) * 1.2
u_L = np.ones(N) * 1.0
u_G = np.ones(N) * 5.0
alpha_L = np.ones(N) * 0.5  # liquid holdup
alpha_G = 1.0 - alpha_L

# Time parameters
dt = 0.001
Nt = 1000

for t in range(Nt):
    # 1. Compute fluxes at cell faces (mass and momentum, each phase)
    uL_face = 0.5 * (u_L[:-1] + u_L[1:])
    uG_face = 0.5 * (u_G[:-1] + u_G[1:])
    alphaL_face = 0.5 * (alpha_L[:-1] + alpha_L[1:])
    alphaG_face = 1.0 - alphaL_face
    rhoL_face = 0.5 * (rho_L[:-1] + rho_L[1:])
    rhoG_face = 0.5 * (rho_G[:-1] + rho_G[1:])

    # Upwind (use limited flux for stability)
    mass_flux_L = flux_limited(alpha_L * rho_L, uL_face) * uL_face * A_total
    mass_flux_G = flux_limited(alpha_G * rho_G, uG_face) * uG_face * A_total

    mom_flux_L = flux_limited(alpha_L * rho_L * u_L, uL_face) * uL_face * A_total
    mom_flux_G = flux_limited(alpha_G * rho_G * u_G, uG_face) * uG_face * A_total

    # 2. Conservative update
    for i in range(1, N-1):
        # Mass
        rho_L[i] -= dt / dx * (mass_flux_L[i] - mass_flux_L[i-1])
        rho_G[i] -= dt / dx * (mass_flux_G[i] - mass_flux_G[i-1])
        # Momentum
        u_L[i] -= dt / dx * (mom_flux_L[i] - mom_flux_L[i-1]) / (rho_L[i] + 1e-8)
        u_G[i] -= dt / dx * (mom_flux_G[i] - mom_flux_G[i-1]) / (rho_G[i] + 1e-8)
    # 3. Artificial viscosity (optional, see Persson-Peraire for details)

    # 4. Simple boundary conditions
    u_L[0] = 1.0; u_G[0] = 5.0
    u_L[-1] = 1.0; u_G[-1] = 5.0

    # Stability check (optional): If np.any(np.isnan(u_L) or np.isnan(u_G)), break/raise

In [3]:
import numpy as np

N = 200
L = 10.0
dx = L / N
r = 0.05
g = 9.81
A_total = np.pi * r**2

# State arrays
alpha_L = np.ones(N) * 0.5 + 0.05 * np.sin(2 * np.pi * np.arange(N) / N)  # perturbation for instability
alpha_G = 1.0 - alpha_L
rho_L = np.ones(N) * 1000.0
rho_G = np.ones(N) * 1.2
u_L = np.ones(N) * 1.0
u_G = np.ones(N) * 5.0

dt = 0.001
Nt = 3000

def compute_geometries(h_int):
    h_int = np.clip(h_int, -r + 1e-8, r - 1e-8)
    r2 = r * r
    root_h = np.sqrt(r2 - h_int**2)
    A_L = r2 * np.arccos(-h_int/r) + h_int * root_h
    A_G = r2 * np.arccos(h_int/r) - h_int * root_h
    P_LG = 2 * root_h
    P_LW = 2*r * np.arccos(-h_int/r)
    P_GW = 2*r * np.arccos(h_int/r)
    return A_L, A_G, P_LG, P_LW, P_GW

def pressure_hydrostatic(p_int, A_L, A_G, rho_L, rho_G, P_LG, h_int):
    p_av_L = p_int + rho_L * g * (h_int + (1/12) * P_LG**3 / (A_L + 1e-12))
    p_av_G = p_int + rho_G * g * (h_int - (1/12) * P_LG**3 / (A_G + 1e-12))
    return p_av_L, p_av_G

# Main loop
for t in range(Nt):
    h_int = (alpha_L - 0.5) * r  # simplistic interface position estimate (refine as needed)
    # 1. Geometry and area/perimeter calculations
    A_L, A_G, P_LG, P_LW, P_GW = compute_geometries(h_int)
    # 2. Pressures
    p_int = 1e5  # could be solved with Poisson eq for full pressure coupling, here use ambient for demo
    p_av_L, p_av_G = pressure_hydrostatic(p_int, A_L, A_G, rho_L, rho_G, P_LG, h_int)
    # 3. Shear terms (use Taitel-Dukler or similar)
    # Skipping for brevity—add drag, wall, and interfacial friction
    
    # 4. Mass fluxes
    mass_flux_L = np.zeros(N+1)
    mass_flux_G = np.zeros(N+1)
    for i in range(1, N):
        mass_flux_L[i] = 0.5 * (A_L[i-1] * rho_L[i-1] * u_L[i-1] + A_L[i] * rho_L[i] * u_L[i])
        mass_flux_G[i] = 0.5 * (A_G[i-1] * rho_G[i-1] * u_G[i-1] + A_G[i] * rho_G[i] * u_G[i])
    
    # 5. Momentum fluxes (include pressure terms and source terms for coupling)
    mom_flux_L = np.zeros(N+1)
    mom_flux_G = np.zeros(N+1)
    for i in range(1, N):
        mom_flux_L[i] = 0.5 * (A_L[i-1] * (rho_L[i-1] * u_L[i-1]**2 + p_av_L[i-1] - p_int) +
                               A_L[i] * (rho_L[i] * u_L[i]**2 + p_av_L[i] - p_int))
        mom_flux_G[i] = 0.5 * (A_G[i-1] * (rho_G[i-1] * u_G[i-1]**2 + p_av_G[i-1] - p_int) +
                               A_G[i] * (rho_G[i] * u_G[i]**2 + p_av_G[i] - p_int))
    
    # 6. Update state (explicit Euler for demo, use implicit for production)
    for i in range(1, N-1):
        # Mass
        d_mass_L = -(mass_flux_L[i+1] - mass_flux_L[i]) / dx
        d_mass_G = -(mass_flux_G[i+1] - mass_flux_G[i]) / dx
        rho_L[i] += dt * d_mass_L / (A_L[i] + 1e-12)
        rho_G[i] += dt * d_mass_G / (A_G[i] + 1e-12)
        # Momentum (including pressure coupling and source terms)
        d_mom_L = -(mom_flux_L[i+1] - mom_flux_L[i]) / dx
        d_mom_G = -(mom_flux_G[i+1] - mom_flux_G[i]) / dx
        # + wall/interfacial shear source terms (add for realism)
        u_L[i] += dt * d_mom_L / (rho_L[i] * A_L[i] + 1e-12)
        u_G[i] += dt * d_mom_G / (rho_G[i] * A_G[i] + 1e-12)

    # Optionally plot alpha_L, u_L, p_av_L every X timesteps

/tmp/ipykernel_41991/1203507868.py:62: RuntimeWarning: overflow encountered in scalar multiply
  A_G[i] * (rho_G[i] * u_G[i]**2 + p_av_G[i] - p_int))
/tmp/ipykernel_41991/1203507868.py:61: RuntimeWarning: overflow encountered in scalar multiply
  mom_flux_G[i] = 0.5 * (A_G[i-1] * (rho_G[i-1] * u_G[i-1]**2 + p_av_G[i-1] - p_int) +
/tmp/ipykernel_41991/1203507868.py:61: RuntimeWarning: invalid value encountered in scalar add
  mom_flux_G[i] = 0.5 * (A_G[i-1] * (rho_G[i-1] * u_G[i-1]**2 + p_av_G[i-1] - p_int) +
/tmp/ipykernel_41991/1203507868.py:73: RuntimeWarning: invalid value encountered in scalar subtract
  d_mom_G = -(mom_flux_G[i+1] - mom_flux_G[i]) / dx
/tmp/ipykernel_41991/1203507868.py:76: RuntimeWarning: invalid value encountered in scalar divide
  u_G[i] += dt * d_mom_G / (rho_G[i] * A_G[i] + 1e-12)


In [20]:
from fenics import *
import numpy as np
import matplotlib.pyplot as plt

from dolfin import set_log_level

set_log_level(30)  # ERROR = 30, WARNING = 20, CRITICAL = 50

N = 200
L = 10.0
mesh = IntervalMesh(N, 0, L)
dt = 1e-4
T = 0.1

V_elem = FiniteElement("DG", mesh.ufl_cell(), 1)
V = FunctionSpace(mesh, V_elem)
W_elem = MixedElement([V_elem]*4)
W = FunctionSpace(mesh, W_elem)

w = Function(W)
w_new = Function(W)
(alpha_L, u_L, alpha_G, u_G) = split(w)
(alpha_L_new, u_L_new, alpha_G_new, u_G_new) = split(w_new)

x = SpatialCoordinate(mesh)
alpha_L0 = project(0.5 + 0.05 * sin(2 * np.pi * x[0] / L), V)
u_L0 = interpolate(Constant(1.0), V)
alpha_G0 = project(1.0 - alpha_L0, V)
u_G0 = interpolate(Constant(5.0), V)

assign(w.sub(0), alpha_L0)
assign(w.sub(1), u_L0)
assign(w.sub(2), alpha_G0)
assign(w.sub(3), u_G0)

rho_L = 1000.0
rho_G = 1.2
R = 287.0
T_gas = 300.0
nu = 1e-2

(v_alphaL, v_uL, v_alphaG, v_uG) = TestFunctions(W)

def flux_alpha_L(alpha_L, u_L): return alpha_L * u_L
def flux_alpha_G(alpha_G, u_G): return alpha_G * u_G
def flux_u_L(alpha_L, u_L): return alpha_L * u_L * u_L
def flux_u_G(alpha_G, u_G): return alpha_G * u_G * u_G

# Pressure as a function of gas phase
def pressure(alpha_G): return alpha_G * rho_G * R * T_gas

F = (
    (alpha_L_new - alpha_L)/dt * v_alphaL * dx
    + flux_alpha_L(alpha_L, u_L).dx(0) * v_alphaL * dx
    + nu * dot(grad(alpha_L_new), grad(v_alphaL)) * dx
    + (u_L_new - u_L)/dt * v_uL * dx
    + flux_u_L(alpha_L, u_L).dx(0) * v_uL * dx
    - alpha_L_new * pressure(alpha_G_new).dx(0) * v_uL * dx
    + nu * dot(grad(u_L_new), grad(v_uL)) * dx
    + (alpha_G_new - alpha_G)/dt * v_alphaG * dx
    + flux_alpha_G(alpha_G, u_G).dx(0) * v_alphaG * dx
    + nu * dot(grad(alpha_G_new), grad(v_alphaG)) * dx
    + (u_G_new - u_G)/dt * v_uG * dx
    + flux_u_G(alpha_G, u_G).dx(0) * v_uG * dx
    - alpha_G_new * pressure(alpha_G_new).dx(0) * v_uG * dx
    + nu * dot(grad(u_G_new), grad(v_uG)) * dx
)

timesteps = int(T/dt)
all_data = np.zeros((timesteps+1, 4, N+1))
for i, var in enumerate(w.split()):
    all_data[0, i, :] = var.compute_vertex_values()

t = 0.0
for step in range(1, timesteps+1):
    solve(F == 0, w_new)
    w.assign(w_new)
    t += dt
    for i, var in enumerate(w.split()):
        all_data[step, i, :] = var.compute_vertex_values()
    # if step % 100 == 0:
    #     plt.clf()
    #     plt.plot(np.linspace(0, L, N+1), all_data[step, 0, :], label="alpha_L")
    #     plt.plot(np.linspace(0, L, N+1), all_data[step, 2, :], label="alpha_G")
    #     plt.legend()
    #     plt.pause(0.01)
# plt.show()